<a href="https://colab.research.google.com/github/RatanakamonS/DADS6003/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Library


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import shap
import matplotlib.pyplot as plt

### 1. Get data and clean data

In [ ]:
# Get data and clean data
url = 'https://raw.githubusercontent.com/RatanakamonS/DADS6003/main/DB-%E0%B8%AA%E0%B8%B3%E0%B8%A3%E0%B8%A7%E0%B8%88%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%83%E0%B8%8A%E0%B9%89%20Smartphone%20(Responses)-Week5%20-%20Form%20Responses.csv'
df = pd.read_csv(url)
df = df.drop(columns=['Timestamp', 'จากข้อที่แล้ว เนื่องจากสาเหตุใด'])
df = df.dropna()
df = df[(df['เพศของผู้ตอบแบบสอบถาม'] != '-') & (df['อาชีพของผู้ตอบแบบสอบถาม'] != '.')]

# เลือก target variable
target = 'ปัจจุบันท่านใช้สมาร์ทโฟนระบบปฎิบัติการใด'

# แปลง target เป็น binary (0: Android, 1: iOS)
df[target] = df[target].map({'Android': 0, 'iOS': 1})

# แยก features และ target
X = df.drop(columns=[target])
y = df[target]

### 2. Train and Test split (70%, 30%)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### 3. Logistic Regression

In [ ]:
# สร้าง pipeline สำหรับ preprocessing และ model
categorical_features = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

### 4.Train Model

In [ ]:
model.fit(X_train, y_train)

### 5. SHAP Explain

### 5.1 Build on X Train

In [ ]:
explainer = shap.LinearExplainer(model['classifier'], model['preprocessor'].transform(X_train))

### 5.2 Explained X Test

In [ ]:
shap_values = explainer(model['preprocessor'].transform(X_test))

### 6. Results

### 6.1 Rank Feature Importance

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

### 6.2 Analyze one sample (Randomly) from test set

In [ ]:
random_index = np.random.randint(0, len(X_test))
random_sample = X_test.iloc[random_index]
shap.force_plot(explainer.expected_value, shap_values[random_index,:], random_sample)

# แสดงผลลัพธ์
print("Model Accuracy:", model.score(X_test, y_test))
print("\nRandom Sample Analysis:")
print(random_sample)
print("\nPrediction:", "iOS" if model.predict(random_sample.to_frame().T)[0] == 1 else "Android")